<h1>LaVague: Quick-tour guide</h1>

In this quick tour we're going to show you how you can use LaVague to perform actions on web from text instructions 🤯

We'll get you started with our two CLI commands:
- `lavague launch`: Launch an interactive in-browser Gradio demo where you can test out instructing LaVague to perform actions on a website!
- `lavague build`: Get the raw Python code generated by LaVague which you can then execute locally

**Pre-requisites:**
- We assume you have already installed LaVague & the necessary dependencies as outlined in our [installation guide](https://docs.lavague.ai/en/latest/docs/get-started/setting-up-la-vague/)
- If you are running the notebook locally, you will need Python (test on python>=3.8) and pip installed
- Our default demo uses the OpenAI API. You will need to have an OPENAI_API_KEY environment variable set in your local environment

> For other integrations, see our [integrations](https://docs.lavague.ai/en/latest/docs/integrations/home/) section

If you prefer to run this example as an end-to-end example with Google Colab, you can follow this link.
<div id="colab_button\">
    <a target="_blank\" href="https://colab.research.google.com/github/lavague-ai/lavague/blob/main/docs/docs/get-started/quick-tour-notebook/quick-tour.ipynb">
    <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Run in Colab"/></a>
    </div>

### LaVague Launch

Let's now run the `lavague launch` command.

- The `--instructions` or `-i` option accepts a text file containing: the URL of the website we will interact with & the instructions for the actions we wish to automate
-  The `--config` or `-c` option with a Python file which can be used to set a desired LLM, embedder etc.

For more info on these, see the [customization guide](./customization.md)!

We will download example of instructions and config before using the launch command:

In [ ]:
# Download files needed for demo to current repo
!wget https://raw.githubusercontent.com/lavague-ai/LaVague/main/examples/configurations/api/openai_api.py
!wget https://raw.githubusercontent.com/lavague-ai/LaVague/main/examples/instructions/huggingface.yaml

We can now run `lavague launch`:

In [ ]:
!lavague -i huggingface.yaml -c openai_api.py launch

You can now click on the public (if you are using Google Colab) or local URL to open your interactive LaVague demo.

⚠️ You'll need to interact with this demo:

1) Click on the URL textbox and press enter. This will show a screenshot of your inital page.

<img src="https://raw.githubusercontent.com/lavague-ai/LaVague/main/docs/assets/launch_1_openai_py.png" />

2) Then select an instruction or write your own, and again click within the instruction textbox and press enter.

At this point, your Python code leveraging Selenium is generated by the LLM & executed. You will see an updated image of the website after the code is executed.

<img src="https://raw.githubusercontent.com/lavague-ai/LaVague/main/docs/assets/launch_2_openai_py.png" />

### LaVague Build

We can use `lavague build` command to generate a file containing this automation code without launching an interactive demo.

In [ ]:
!lavague -i huggingface.yaml -c openai_api.py build

This creates a script in your current directory named `hf_openai_api_gen.py` - a combination of the two input file names separated by a `_` character with a `_gen` suffix.

We can now inspect the code and execute it locally!

<img src="https://raw.githubusercontent.com/lavague-ai/LaVague/main/docs/assets/build_openai_py.png"/>


If you have any further questions, join us on the LaVague Discord [here](https://discord.com/invite/SDxn9KpqX9).